In [1]:
# import sys
# import os
# sys.dont_write_bytecode = True
# os.environ["NUMBA_DISABLE_JIT"] = "1"

In [2]:
import numpy as np

from my_stuff import MufexKeys
import requests
from quantfreedom.exchanges.mufex_exchange.mufex import Mufex
from quantfreedom.enums import DynamicOrderSettingsArrays
from quantfreedom.enums import *
from quantfreedom.helper_funcs import dos_cart_product
from quantfreedom.nb_funcs.nb_strategies.nb_strategy import (
    nb_strat_bt_create_ind,
    nb_strat_evaluate,
    nb_strat_get_current_ind_settings,
    nb_strat_get_ind_set_str,
    nb_strat_get_total_ind_settings,
)
from quantfreedom.nb_funcs.nb_simulate import nb_sim_df_backtest
from quantfreedom.nb_funcs.nb_sim_order_records import nb_sim_get_or


mufex_main = Mufex(
    api_key=MufexKeys.api_key,
    secret_key=MufexKeys.secret_key,
    use_test_net=False,
)

%load_ext autoreload
%autoreload 2

In [3]:
candles_np = mufex_main.get_candles(
    symbol="BTCUSDT", timeframe="5m", candles_to_dl=4000)

In [5]:
mufex_main.set_exchange_settings(
    symbol="BTCUSDT",
    position_mode=PositionModeType.HedgeMode,
    leverage_mode=LeverageModeType.Isolated,
)

backtest_settings = BacktestSettings(
    array_size=1000,
    gains_pct_filter=0,
    total_trade_filter=40,
    upside_filter=0,
)

# backtest_settings = BacktestSettings()

dos_arrays = DynamicOrderSettingsArrays(
    max_equity_risk_pct=np.array([2, 3, 4, 5, 6]) * 3,
    max_trades=np.array([5]),
    num_candles=np.array([100]),
    risk_account_pct_size=np.array([3]),
    risk_reward=np.array([2, 4, 6, 10]),
    sl_based_on_add_pct=np.array([0.01, 0.05, 0.1]),
    sl_based_on_lookback=np.array([20, 40, 60]),
    sl_bcb_type=np.array([CandleBodyType.Low]),
    sl_to_be_cb_type=np.array([CandleBodyType.Nothing]),
    sl_to_be_when_pct=np.array([0]),
    static_leverage=np.array([0]),
    trail_sl_bcb_type=np.array([CandleBodyType.Low]),
    trail_sl_by_pct=np.array([0.2, 0.5, 1.0, 1.5]),
    trail_sl_when_pct=np.array([0.2, 0.4, 0.8, 1, 2]),
)

static_os = StaticOrderSettings(
    increase_position_type=IncreasePositionType.RiskPctAccountEntrySize,
    leverage_strategy_type=LeverageStrategyType.Dynamic,
    logger_bool=False,
    long_or_short=LongOrShortType.Long,
    pg_min_max_sl_bcb=PriceGetterType.Min,
    sl_strategy_type=StopLossStrategyType.SLBasedOnCandleBody,
    sl_to_be_bool=False,
    starting_equity=1000.0,
    tp_fee_type=TakeProfitFeeType.Limit,
    tp_strategy_type=TakeProfitStrategyType.RiskReward,
    trail_sl_bool=True,
    z_or_e_type=ZeroOrEntryType.Nothing,
)

In [6]:
strat_records_df = nb_sim_df_backtest(
    backtest_settings=backtest_settings,
    candles=candles_np,
    dos_arrays=dos_arrays,
    evaluate=nb_strat_evaluate,
    exchange_settings=mufex_main.exchange_settings,
    get_current_ind_settings=nb_strat_get_current_ind_settings,
    get_ind_set_str=nb_strat_get_ind_set_str,
    get_total_ind_settings=nb_strat_get_total_ind_settings,
    ind_creator=nb_strat_bt_create_ind,
    nb_logger_type=NBLoggerType.Pass,
    starting_equity=1000.0,
    static_os=static_os,
)
strat_records_df

Starting the backtest now ... and also here are some stats for your backtest.

Total indicator settings to test: 6
Total order settings to test: 3,600
Total combinations of settings to test: 21,600
Total candles: 4,000
Total candles to test: 86,400,000


,ind_set_idx,dos_index,total_trades,wins,losses,gains_pct,win_rate,to_the_upside,fees_paid,total_pnl,ending_eq
0,0,102,54.0,27,27,0.052,50.000,0.031,488.698,0.523,1000.523
1,0,309,54.0,12,42,1.572,22.222,0.334,736.897,15.724,1015.724
2,0,439,42.0,9,33,16.404,21.429,0.242,636.011,164.037,1164.037
3,0,495,48.0,9,39,34.455,18.750,0.259,673.385,344.547,1344.547
4,0,496,48.0,9,39,34.455,18.750,0.259,673.385,344.547,1344.547
...,...,...,...,...,...,...,...,...,...,...,...
80,5,1036,44.0,9,35,17.162,20.455,0.059,592.939,171.617,1171.617
81,5,1037,44.0,9,35,17.162,20.455,0.059,592.939,171.617,1171.617
82,5,1038,44.0,9,35,17.162,20.455,0.059,592.939,171.617,1171.617
83,5,1094,42.0,9,33,4.821,21.429,0.526,807.762,48.210,1048.210


In [7]:
strat_records_df.sort_values(by='to_the_upside', ascending=False)

,ind_set_idx,dos_index,total_trades,wins,losses,gains_pct,win_rate,to_the_upside,fees_paid,total_pnl,ending_eq
61,4,194,44.0,11,33,27.550,25.000,0.630,781.853,275.500,1275.500
75,5,194,42.0,10,32,28.001,23.810,0.627,746.097,280.010,1280.010
74,5,189,46.0,11,35,18.451,23.913,0.621,790.657,184.511,1184.511
84,5,1634,42.0,9,33,10.772,21.429,0.615,1103.738,107.725,1107.725
66,4,369,44.0,9,35,9.511,20.455,0.592,812.212,95.109,1095.109
...,...,...,...,...,...,...,...,...,...,...,...
68,4,1036,44.0,10,34,7.202,22.727,0.002,567.401,72.015,1072.015
67,4,1035,44.0,10,34,7.202,22.727,0.002,567.401,72.015,1072.015
70,4,1038,44.0,10,34,7.202,22.727,0.002,567.401,72.015,1072.015
43,2,343,44.0,20,24,2.507,45.455,0.001,340.636,25.071,1025.071


In [10]:
order_records_df, data = nb_sim_get_or(
    backtest_settings=backtest_settings,
    candles=candles_np,
    dos_arrays=dos_arrays,
    evaluate=nb_strat_evaluate,
    exchange_settings=mufex_main.exchange_settings,
    get_current_ind_settings=nb_strat_get_current_ind_settings,
    get_ind_set_str=nb_strat_get_ind_set_str,
    get_total_ind_settings=nb_strat_get_total_ind_settings,
    ind_creator=nb_strat_bt_create_ind,
    nb_logger_type=NBLoggerType.Pass,
    starting_equity=1000.0,
    static_os=static_os,
    ind_set_index=4,
    dos_index=194,
    plot_results=True,
)

In [ ]:
# res = requests.post("http://127.0.0.1:7779/plot", json=data)

# returned_data = res.json()

# print(returned_data)